# Part 06b: Visualize seattle street groups by type
michael babb  
2024 11 24

In [57]:
# standard
import os

In [58]:
# external
import geopandas as gpd
import matplotlib as mpl
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [59]:
# custom
import run_constants as rc
from utils import *

# load the working seattle street network data

In [60]:
fpn = os.path.join(rc.OUTPUT_FILE_PATH, rc.S03_SND_WORKING_IN_FILE_NAME)

In [61]:
gdf = gpd.read_file(filename = fpn)

In [62]:
gdf.shape

(25773, 38)

In [63]:
gdf['ord_stname_concat'].unique().shape

(2455,)

In [66]:
gdf.columns

Index(['f_intr_id', 't_intr_id', 'snd_id', 'snd_feacode', 'citycode',
       'stname_id', 'st_code', 'arterial_code', 'segment_type', 'agency_code',
       'access_code', 'divided_code', 'structure_type', 'legalloc_code',
       'vehicle_use_code', 'gis_seg_length', 'l_adrs_from', 'l_adrs_to',
       'r_adrs_from', 'r_adrs_to', 'ord_pre_dir', 'ord_street_name',
       'ord_street_type', 'ord_suf_dir', 'ord_stname_concat', 'l_city',
       'l_state', 'l_zip', 'r_city', 'r_state', 'r_zip', 'sndseg_update',
       'compkey', 'comptype', 'unitid', 'unitid2', 'city_portion', 'geometry'],
      dtype='object')

In [65]:
gdf['city_portion'].unique()

array(['S', 'E', 'NW', 'N', 'CNTR', 'NE', 'SW', 'W'], dtype=object)

## load the manually created street group data
These data were created in Step 6, but we'll bring them into the Step 3 workflow.

In [67]:
fpn = os.path.join(rc.INPUT_FILE_PATH, rc.S03_STREET_GROUP_IN_FILE_NAME)

In [68]:
index_cols = ['sort_order', 'ord_street_name', 'ord_street_type', 'city_portion']
sg_df = pd.read_excel(io = fpn, index_col=[0, 1, 2, 3])
sg_df = sg_df.reset_index()

In [69]:
drop_cols = ['sort_order', 'progress', 'count']
sg_df = sg_df.drop(labels = drop_cols, axis = 1)


In [11]:
test_join = pd.merge(left = gdf, right = sg_df, how = 'outer', indicator=True)
test_join['_merge'].value_counts()

_merge
both          19032
left_only      6741
right_only        7
Name: count, dtype: int64

In [12]:
gdf = test_join.copy()

In [13]:
gdf['group_id'] = gdf['group_id'].fillna(1).astype(int)
gdf.shape

(25780, 40)

# create an id across street groups

In [14]:
gdf.head()

,f_intr_id,t_intr_id,snd_id,snd_feacode,citycode,stname_id,st_code,arterial_code,segment_type,agency_code,...,r_zip,sndseg_update,compkey,comptype,unitid,unitid2,city_portion,geometry,group_id,_merge
0,3836.0,3893.0,4787.0,5.0,1.0,1955.0,0.0,1.0,1.0,1.0,...,98177,2011-09-09 00:00:00+00:00,18112.0,68.0,14050,0040,NW,"LINESTRING (-122.36206 47.7035, -122.36342 47....",1,both
1,3893.0,3906.0,4802.0,5.0,1.0,1955.0,0.0,1.0,1.0,1.0,...,98177,2011-09-09 00:00:00+00:00,18113.0,68.0,14050,0043,NW,"LINESTRING (-122.36342 47.70275, -122.36386 47...",1,both
2,3906.0,3973.0,4864.0,5.0,1.0,1955.0,0.0,1.0,1.0,1.0,...,98177,2019-08-13 14:00:50+00:00,18114.0,68.0,14050,0070,NW,"LINESTRING (-122.36386 47.70251, -122.36535 47...",1,both
3,48444.0,48443.0,49871.0,1.0,0.0,3767.0,0.0,0.0,1.0,1.0,...,98146,2022-07-19 16:58:07+00:00,0.0,0.0,0,0,SW,"LINESTRING (-122.3403 47.51334, -122.34036 47....",2,both
4,4014.0,4015.0,4911.0,1.0,1.0,1465.0,0.0,0.0,1.0,1.0,...,98133,2005-05-04 00:00:00+00:00,14968.0,68.0,11695,0110,N,"LINESTRING (-122.34467 47.70144, -122.34197 47...",1,both


In [15]:
gdf.columns

Index(['f_intr_id', 't_intr_id', 'snd_id', 'snd_feacode', 'citycode',
       'stname_id', 'st_code', 'arterial_code', 'segment_type', 'agency_code',
       'access_code', 'divided_code', 'structure_type', 'legalloc_code',
       'vehicle_use_code', 'gis_seg_length', 'l_adrs_from', 'l_adrs_to',
       'r_adrs_from', 'r_adrs_to', 'ord_pre_dir', 'ord_street_name',
       'ord_street_type', 'ord_suf_dir', 'ord_stname_concat', 'l_city',
       'l_state', 'l_zip', 'r_city', 'r_state', 'r_zip', 'sndseg_update',
       'compkey', 'comptype', 'unitid', 'unitid2', 'city_portion', 'geometry',
       'group_id', '_merge'],
      dtype='object')

In [16]:
col_names = ['ord_street_name', 'ord_street_type',  'ord_stname_concat', 'city_portion', 'group_id']
id_df = gdf[col_names].drop_duplicates()

In [17]:
id_df['ord_stname_type'] = id_df['ord_street_name'] + ' ' + id_df['ord_street_type']

In [18]:
id_df['sort_order'] = id_df['ord_street_name'].map(get_sort_order)

In [19]:
id_df = id_df.sort_values(by = ['sort_order', 'ord_street_type', 'city_portion'])

In [20]:
id_df.head()

,ord_street_name,ord_street_type,ord_stname_concat,city_portion,group_id,ord_stname_type,sort_order
3387,1ST,AVE,1ST AVE,CNTR,2,1ST AVE,001
3412,1ST,AVE,1ST AVE N,N,2,1ST AVE,001
3449,1ST,AVE,1ST AVE NE,NE,2,1ST AVE,001
3504,1ST,AVE,1ST AVE NW,NW,1,1ST AVE,001
3578,1ST,AVE,1ST AVE S,S,2,1ST AVE,001


In [21]:
id_df['ord_stname_concat'].unique().shape

(2456,)

In [22]:
# let's get the groups! can we aggregate this?

In [23]:
col_names = ['sort_order', 'ord_street_name', 'ord_street_type', 'ord_stname_type', 'group_id', 'city_portion']
test_agg = id_df[col_names].groupby(col_names[:-1]).agg(city_portion_group = ('city_portion', lambda x: '_'.join(sorted(set(x)))),
                                                        n_groups = ('city_portion', lambda x: len(set(x)))).reset_index()

In [24]:
test_agg.head()

,sort_order,ord_street_name,ord_street_type,ord_stname_type,group_id,city_portion_group,n_groups
0,001,1ST,AVE,1ST AVE,1,NW_SW_W,3
1,001,1ST,AVE,1ST AVE,2,CNTR_N_NE_S,4
2,001,1ST,PL,1ST PL,1,NE,1
3,002,2ND,AVE,2ND AVE,1,NW_SW_W,3
4,002,2ND,AVE,2ND AVE,2,CNTR_N_NE_S,4


In [25]:
test_agg['city_portion_group'].unique().shape

(33,)

In [26]:
test_agg.head()

,sort_order,ord_street_name,ord_street_type,ord_stname_type,group_id,city_portion_group,n_groups
0,001,1ST,AVE,1ST AVE,1,NW_SW_W,3
1,001,1ST,AVE,1ST AVE,2,CNTR_N_NE_S,4
2,001,1ST,PL,1ST PL,1,NE,1
3,002,2ND,AVE,2ND AVE,1,NW_SW_W,3
4,002,2ND,AVE,2ND AVE,2,CNTR_N_NE_S,4


In [27]:
test_cpg = 'CNTR_W'
test_ost = 'BR'
check_df = test_agg.loc[(test_agg['city_portion_group'] == test_cpg) &
                        (test_agg['ord_street_type'] == test_ost), ]
print(check_df.shape)
check_df.head()

(1, 7)


,sort_order,ord_street_name,ord_street_type,ord_stname_type,group_id,city_portion_group,n_groups
1251,MAGNOLIA,MAGNOLIA,BR,MAGNOLIA BR,1,CNTR_W,2


In [28]:
# export to excel
city_portion_check = test_agg[['city_portion_group', 'ord_street_type', 'n_groups']].drop_duplicates().reset_index(drop = True)

In [29]:
city_portion_check.head()

,city_portion_group,ord_street_type,n_groups
0,NW_SW_W,AVE,3
1,CNTR_N_NE_S,AVE,4
2,NE,PL,1
3,SW,LN,1
4,SW,PL,1


In [30]:
city_portion_check.shape

(129, 3)

In [31]:
city_portion_check = city_portion_check.sort_values(by = ['city_portion_group', 'ord_street_type'])

In [32]:
city_portion_check.to_excel(excel_writer='city_portion_check.xlsx', index = False)

In [33]:
test_agg.head()

,sort_order,ord_street_name,ord_street_type,ord_stname_type,group_id,city_portion_group,n_groups
0,001,1ST,AVE,1ST AVE,1,NW_SW_W,3
1,001,1ST,AVE,1ST AVE,2,CNTR_N_NE_S,4
2,001,1ST,PL,1ST PL,1,NE,1
3,002,2ND,AVE,2ND AVE,1,NW_SW_W,3
4,002,2ND,AVE,2ND AVE,2,CNTR_N_NE_S,4


In [34]:
test_agg.shape

(1816, 7)

In [35]:
# join back to the working gdf

In [36]:
gdf.columns

Index(['f_intr_id', 't_intr_id', 'snd_id', 'snd_feacode', 'citycode',
       'stname_id', 'st_code', 'arterial_code', 'segment_type', 'agency_code',
       'access_code', 'divided_code', 'structure_type', 'legalloc_code',
       'vehicle_use_code', 'gis_seg_length', 'l_adrs_from', 'l_adrs_to',
       'r_adrs_from', 'r_adrs_to', 'ord_pre_dir', 'ord_street_name',
       'ord_street_type', 'ord_suf_dir', 'ord_stname_concat', 'l_city',
       'l_state', 'l_zip', 'r_city', 'r_state', 'r_zip', 'sndseg_update',
       'compkey', 'comptype', 'unitid', 'unitid2', 'city_portion', 'geometry',
       'group_id', '_merge'],
      dtype='object')

In [37]:
gdf = gdf.drop(labels = ['_merge'], axis = 1)

In [38]:
test_join = pd.merge(left = gdf, right = test_agg, how = 'outer', indicator=True)

In [39]:
test_join.shape

(25780, 44)

In [40]:
test_join.columns

Index(['f_intr_id', 't_intr_id', 'snd_id', 'snd_feacode', 'citycode',
       'stname_id', 'st_code', 'arterial_code', 'segment_type', 'agency_code',
       'access_code', 'divided_code', 'structure_type', 'legalloc_code',
       'vehicle_use_code', 'gis_seg_length', 'l_adrs_from', 'l_adrs_to',
       'r_adrs_from', 'r_adrs_to', 'ord_pre_dir', 'ord_street_name',
       'ord_street_type', 'ord_suf_dir', 'ord_stname_concat', 'l_city',
       'l_state', 'l_zip', 'r_city', 'r_state', 'r_zip', 'sndseg_update',
       'compkey', 'comptype', 'unitid', 'unitid2', 'city_portion', 'geometry',
       'group_id', 'sort_order', 'ord_stname_type', 'city_portion_group',
       'n_groups', '_merge'],
      dtype='object')

In [41]:
test_join['_merge'].value_counts()

_merge
both          25780
left_only         0
right_only        0
Name: count, dtype: int64

In [42]:
gdf = test_join.copy()

# MAKE A PLOT OF THE DIFFERENT CITY GROUPS BY STREET TYPE

In [43]:
# use the same colors for each city portion
cdm = {}
dir_list = ['CNTR', 'E', 'N', 'NE', 'NW', 'S', 'SW', 'W']
for idir, dir_value in enumerate(dir_list):
    cdm[dir_value] = mpl.colormaps["Dark2"].colors[idir]

my_cmap = mpl.colors.ListedColormap([cdm[c] for c in dir_list])

In [44]:
wgdf = gdf.loc[gdf['n_groups'] > 1, :].copy()

In [45]:
wgdf.shape

(18205, 44)

In [46]:
col_names = ['city_portion_group', 'ord_street_type']
control_df = wgdf[col_names].drop_duplicates()

In [47]:
control_df.shape

(44, 2)

In [48]:
control_df.head()

,city_portion_group,ord_street_type
4,N_NE_NW,ST
73,S_SW,ST
126,N_NW,ST
144,N_NE,ST
561,NE_NW,ST


In [49]:
# plot bounds to use across each plot
bounds = [-122.4197794277490061,47.4803548409661005, -122.2200188105690017,47.7341482423694004]

In [50]:
# load the boundaries of the city sections
ifpn = os.path.join(rc.OUTPUT_FILE_PATH, rc.S02_CITY_SECTIONS_OUT_FILE_NAME)
cs_gdf = gpd.read_file(filename=ifpn)

In [51]:
control_df.head()

,city_portion_group,ord_street_type
4,N_NE_NW,ST
73,S_SW,ST
126,N_NW,ST
144,N_NE,ST
561,NE_NW,ST


In [53]:
ost = 'BR'
pgdf = gdf.loc[(gdf['ord_street_type'] == ost), :]

In [55]:
pgdf['city_portion_group'].unique()

array(['S', 'CNTR_N', 'NW_W', 'CNTR', 'CNTR_W', 'E', 'SW', 'E_NE'],
      dtype=object)

In [54]:
pgdf.head()

,f_intr_id,t_intr_id,snd_id,snd_feacode,citycode,stname_id,st_code,arterial_code,segment_type,agency_code,...,unitid,unitid2,city_portion,geometry,group_id,sort_order,ord_stname_type,city_portion_group,n_groups,_merge
2947,47553.0,17931.0,38578.0,5.0,0.0,2870.0,0.0,1.0,1.0,1.0,...,00705,0820,S,"LINESTRING (-122.31449 47.52947, -122.31456 47...",1,016,16TH AV S BR,S,1,both
14143,9358.0,9107.0,16776.0,5.0,1.0,2966.0,1.0,1.0,1.0,1.0,...,05254,0258,CNTR,"LINESTRING (-122.34717 47.643, -122.34719 47.6...",1,AURORA,AURORA BR,CNTR_N,2,both
14144,8947.0,8739.0,11489.0,5.0,1.0,2966.0,1.0,1.0,1.0,1.0,...,05254,0258,N,"LINESTRING (-122.34732 47.64855, -122.34732 47...",1,AURORA,AURORA BR,CNTR_N,2,both
14145,9107.0,8947.0,16480.0,5.0,1.0,2966.0,0.0,1.0,1.0,1.0,...,05254,0258,N,"LINESTRING (-122.3473 47.64604, -122.34731 47....",1,AURORA,AURORA BR,CNTR_N,2,both
14236,7743.0,7517.0,10647.0,5.0,1.0,2974.0,0.0,1.0,1.0,1.0,...,05315,0440,NW,"LINESTRING (-122.37624 47.66008, -122.37624 47...",1,BALLARD,BALLARD BR,NW_W,2,both


In [52]:
make_plots = True
if make_plots:
    for cr, crow in control_df.iterrows():
        cpg = crow['city_portion_group']
        ost = crow['ord_street_type']

        output_file_name = f"{ost}_{cpg}.png"
        ofpn = os.path.join(rc.S06_PLOT_OUTPUT_FILE_PATH_CITY_PORTION_GROUPS, output_file_name)
        print(ofpn)

        # now, let's make a map...
        pgdf = gdf.loc[(gdf['city_portion_group'] == cpg) &
                    (gdf['ord_street_type'] == ost), :]
        to_draw = pgdf[['city_portion', 'geometry']].dissolve(by = 'city_portion', as_index = False)
        #to_draw['coords'] = to_draw['geometry'].map(lambda x: x.centroid.coords[0])
        fig = plt.figure(layout = 'constrained', figsize = (5, 10))
        gs = GridSpec(1,1, figure = fig, height_ratios = [1])
        ax1 = fig.add_subplot(gs[0,0])
        ax1.set_xlim(bounds[0], bounds[2])
        ax1.set_ylim(bounds[1], bounds[3])
        cs_gdf.plot(ax = ax1, column = 'city_portion', alpha = .2)
        to_draw.plot(ax = ax1, column = 'city_portion', cmap = my_cmap,  linewidth = 5, legend = True)

        #for irrow, row in to_draw.iterrows():    
        #    ax1.annotate(text=row['city_portion'], xy=row['coords'], fontsize = 16 )

        ax1.set_axis_off()
        
        my_title = f"Street Type: {ost} | City Portion Group {cpg}"
        plt.title(label = my_title)

        #plt.show()

        fig.savefig(fname = ofpn)
        plt.close()

../../../project/seattle_streets/print/city_portion_groups\ST_N_NE_NW.png
../../../project/seattle_streets/print/city_portion_groups\ST_S_SW.png
../../../project/seattle_streets/print/city_portion_groups\ST_N_NW.png
../../../project/seattle_streets/print/city_portion_groups\ST_N_NE.png
../../../project/seattle_streets/print/city_portion_groups\ST_NE_NW.png
../../../project/seattle_streets/print/city_portion_groups\AVE_NW_SW_W.png
../../../project/seattle_streets/print/city_portion_groups\AVE_CNTR_E_NE_S.png
../../../project/seattle_streets/print/city_portion_groups\AVE_CNTR_E_S.png
../../../project/seattle_streets/print/city_portion_groups\AVE_CNTR_N_NE_S.png
../../../project/seattle_streets/print/city_portion_groups\AVE_SW_W.png
../../../project/seattle_streets/print/city_portion_groups\AVE_E_NE_S.png
../../../project/seattle_streets/print/city_portion_groups\AVE_NE_S.png
../../../project/seattle_streets/print/city_portion_groups\WAY_CNTR_S_W.png
../../../project/seattle_streets/print